In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torchtext
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import spacy
import numpy as np
import pandas as pd
import random
import math
import time
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Step 1: Load the Dataset

In [ ]:
data_df = pd.read_csv(
    'C:\\Users\\Manish Acharya\\OneDrive\\Desktop\\Proj-2\\transformer\\spoc-train.tsv', delimiter='\t')

Step 2: Handle Missing Values<br>
Drop rows with missing values in 'text' or 'code'

In [ ]:
data_df = data_df.dropna(subset=['text', 'code'])

Step 3: Tokenization and Vocabulary Creation

In [ ]:
tokenizer = Tokenizer(filters='', lower=False, oov_token='<UNK>')
combined_text = list(data_df['text']) + list(data_df['code'])
tokenizer.fit_on_texts(combined_text)

Vocabulary mapping: token to index

In [ ]:
vocab = tokenizer.word_index
# Add special tokens to the vocabulary
special_tokens = {'<PAD>': 0, '<SOS>': len(vocab) + 1, '<EOS>': len(vocab) + 2}
vocab.update(special_tokens)

Padding Index

In [ ]:
SRC_PAD_IDX = vocab['<PAD>']  # The padding index for input (pseudocode)
TRG_PAD_IDX = vocab['<PAD>']  # The padding index for output (CPP code)

Reverse vocabulary mapping: index to token

In [ ]:
reverse_vocab = {index: token for token, index in vocab.items()}

Input and Output Dimensions

In [ ]:
INPUT_DIM = len(vocab)  # Vocabulary size for pseudocode
OUTPUT_DIM = len(vocab)  # Vocabulary size for CPP code

Step 4: Numerical Representation and Padding

In [ ]:
pseudocode_sequences = tokenizer.texts_to_sequences(data_df['text'])
cpp_code_sequences = tokenizer.texts_to_sequences(data_df['code'])

Pad sequences to have the same length

In [ ]:
max_sequence_length = max(len(seq)
                          for seq in pseudocode_sequences + cpp_code_sequences)
padded_pseudocode = pad_sequences(
    pseudocode_sequences, maxlen=max_sequence_length, padding='post')
padded_cpp_code = pad_sequences(
    cpp_code_sequences, maxlen=max_sequence_length, padding='post')

Step 5: Data Splitting

In [ ]:
train_pseudocode, val_test_pseudocode, train_cpp_code, val_test_cpp_code = train_test_split(
    padded_pseudocode, padded_cpp_code, test_size=0.2, random_state=42
)
val_pseudocode, test_pseudocode, val_cpp_code, test_cpp_code = train_test_split(
    val_test_pseudocode, val_test_cpp_code, test_size=0.5, random_state=42
)

Step 6: Save Preprocessed Data and Vocabulary

In [ ]:
with open('vocab.txt', 'w') as file:
    for token, index in vocab.items():
        file.write(f"{token}\t{index}\n")

In [ ]:
np.save('train_pseudocode.npy', train_pseudocode)
np.save('val_pseudocode.npy', val_pseudocode)
np.save('test_pseudocode.npy', test_pseudocode)
np.save('train_cpp_code.npy', train_cpp_code)
np.save('val_cpp_code.npy', val_cpp_code)
np.save('test_cpp_code.npy', test_cpp_code)

Check if GPU is available and use it, otherwise fall back to CPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Step 7: Define the Transformer Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim,
                 hid_dim,
                 n_layers,
                 n_heads,
                 pf_dim,
                 dropout,
                 device,
                 max_length=100):
        super().__init__()
        self.device = device
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        self.layers = nn.ModuleList([EncoderLayer(hid_dim,
                                                  n_heads,
                                                  pf_dim,
                                                  dropout,
                                                  device)
                                    for _ in range(n_layers)])
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
    def forward(self, src, src_mask):

        # src = [batch size, src len]
        # src_mask = [batch size, 1, 1, src len]
        batch_size = src.shape[0]
        src_len = src.shape[1]
        pos = torch.arange(0, src_len).unsqueeze(
            0).repeat(batch_size, 1).to(self.device)

        # pos = [batch size, src len]
        src = self.dropout(
            (self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

        # src = [batch size, src len, hid dim]
        for layer in self.layers:
            src = layer(src, src_mask)

            # src = [batch size, src len, hid dim]
        return src

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self,
                 hid_dim,
                 n_heads,
                 pf_dim,
                 dropout,
                 device):
        super().__init__()
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(
            hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim,
                                                                     pf_dim,
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
    def forward(self, src, src_mask):

        # src = [batch size, src len, hid dim]
        # src_mask = [batch size, 1, 1, src len]

        # self attention
        _src, _ = self.self_attention(src, src, src, src_mask)

        # dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        # src = [batch size, src len, hid dim]

        # positionwise feedforward
        _src = self.positionwise_feedforward(src)

        # dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))

        # src = [batch size, src len, hid dim]
        return src

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):

        # x = [batch size, seq len, hid dim]
        x = self.dropout(torch.relu(self.fc_1(x)))

        # x = [batch size, seq len, pf dim]
        x = self.fc_2(x)

        # x = [batch size, seq len, hid dim]
        return x

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        assert hid_dim % n_heads == 0
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(
            torch.FloatTensor([self.head_dim])).to(device)
    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]

        # query = [batch size, query len, hid dim]
        # key = [batch size, key len, hid dim]
        # value = [batch size, value len, hid dim]
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        # Q = [batch size, query len, hid dim]
        # K = [batch size, key len, hid dim]
        # V = [batch size, value len, hid dim]
        Q = Q.view(batch_size, -1, self.n_heads,
                   self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads,
                   self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads,
                   self.head_dim).permute(0, 2, 1, 3)

        # Q = [batch size, n heads, query len, head dim]
        # K = [batch size, n heads, key len, head dim]
        # V = [batch size, n heads, value len, head dim]
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        # energy = [batch size, n heads, query len, key len]
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        attention = torch.softmax(energy, dim=-1)

        # attention = [batch size, n heads, query len, key len]
        x = torch.matmul(self.dropout(attention), V)

        # x = [batch size, n heads, query len, head dim]
        x = x.permute(0, 2, 1, 3).contiguous()

        # x = [batch size, query len, n heads, head dim]
        x = x.view(batch_size, -1, self.hid_dim)

        # x = [batch size, query len, hid dim]
        x = self.fc_o(x)

        # x = [batch size, query len, hid dim]
        return x, attention

In [ ]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim,
                 hid_dim,
                 n_layers,
                 n_heads,
                 pf_dim,
                 dropout,
                 device,
                 max_length=100):
        super().__init__()
        self.device = device
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        self.layers = nn.ModuleList([DecoderLayer(hid_dim,
                                                  n_heads,
                                                  pf_dim,
                                                  dropout,
                                                  device)
                                     for _ in range(n_layers)])
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg = [batch size, trg len]
        # enc_src = [batch size, src len, hid dim]
        # trg_mask = [batch size, 1, trg len, trg len]
        # src_mask = [batch size, 1, 1, src len]
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        pos = torch.arange(0, trg_len).unsqueeze(
            0).repeat(batch_size, 1).to(self.device)

        # pos = [batch size, trg len]
        trg = self.dropout(
            (self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

        # trg = [batch size, trg len, hid dim]
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)

        # trg = [batch size, trg len, hid dim]
        # attention = [batch size, n heads, trg len, src len]
        output = self.fc_out(trg)

        # output = [batch size, trg len, output dim]
        return output, attention

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self,
                 hid_dim,
                 n_heads,
                 pf_dim,
                 dropout,
                 device):
        super().__init__()
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(
            hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(
            hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim,
                                                                     pf_dim,
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg = [batch size, trg len, hid dim]
        # enc_src = [batch size, src len, hid dim]
        # trg_mask = [batch size, 1, trg len, trg len]
        # src_mask = [batch size, 1, 1, src len]

        # self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

        # dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

        # trg = [batch size, trg len, hid dim]

        # encoder attention
        _trg, attention = self.encoder_attention(
            trg, enc_src, enc_src, src_mask)
        # query, key, value

        # dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

        # trg = [batch size, trg len, hid dim]

        # positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        # dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        # trg = [batch size, trg len, hid dim]
        # attention = [batch size, n heads, trg len, src len]
        return trg, attention

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder,
                 decoder,
                 src_pad_idx,
                 trg_pad_idx,
                 device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
    def make_src_mask(self, src):

        # src = [batch size, src len]
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask = [batch size, 1, 1, src len]
        return src_mask
    def make_trg_mask(self, trg):

        # trg = [batch size, trg len]
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

        # trg_pad_mask = [batch size, 1, 1, trg len]
        trg_len = trg.shape[1]
        trg_sub_mask = torch.tril(torch.ones(
            (trg_len, trg_len), device=self.device)).bool()

        # trg_sub_mask = [trg len, trg len]
        trg_mask = trg_pad_mask & trg_sub_mask

        # trg_mask = [batch size, 1, trg len, trg len]
        return trg_mask
    def forward(self, src, trg):

        # src = [batch size, src len]
        # trg = [batch size, trg len]
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # src_mask = [batch size, 1, 1, src len]
        # trg_mask = [batch size, 1, trg len, trg len]
        enc_src = self.encoder(src, src_mask)

        # enc_src = [batch size, src len, hid dim]
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        # output = [batch size, trg len, output dim]
        # attention = [batch size, n heads, trg len, src len]
        return output, attention

In [ ]:
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 16
DEC_HEADS = 16
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

Build the Transformer Model

In [ ]:
enc = Encoder(INPUT_DIM, HID_DIM, ENC_LAYERS, ENC_HEADS,
              ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HID_DIM, DEC_LAYERS,
              DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)
model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

Step 8: Load Preprocessed Data and Vocabulary

In [ ]:
train_pseudocode = np.load('train_pseudocode.npy')
val_pseudocode = np.load('val_pseudocode.npy')
test_pseudocode = np.load('test_pseudocode.npy')
train_cpp_code = np.load('train_cpp_code.npy')
val_cpp_code = np.load('val_cpp_code.npy')
test_cpp_code = np.load('test_cpp_code.npy')

In [ ]:
vocab = {}
with open('vocab.txt', 'r') as file:
    for line in file:
        token, index = line.strip().split('\t')
        vocab[token] = int(index)
vocab_size = len(vocab)

Add special tokens

In [ ]:
PAD_token = vocab['<PAD>']
SOS_token = vocab['<SOS>']
EOS_token = vocab['<EOS>']

Step 9: Create PyTorch Dataset and DataLoader

In [ ]:
class CodeGenerationDataset(Dataset):
    def __init__(self, pseudocode_sequences, cpp_code_sequences):
        self.pseudocode_sequences = pseudocode_sequences
        self.cpp_code_sequences = cpp_code_sequences
    def __len__(self):
        return len(self.pseudocode_sequences)
    def __getitem__(self, index):
        pseudocode = torch.tensor(
            self.pseudocode_sequences[index], dtype=torch.long)
        cpp_code = torch.tensor(
            self.cpp_code_sequences[index], dtype=torch.long)
        return pseudocode, cpp_code

Create datasets and data loaders

In [ ]:
batch_size = 64
train_dataset = CodeGenerationDataset(train_pseudocode, train_cpp_code)
val_dataset = CodeGenerationDataset(val_pseudocode, val_cpp_code)
test_dataset = CodeGenerationDataset(test_pseudocode, test_cpp_code)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Step 10: "Train the model"

Define your model hyperparameters

In [ ]:
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 16
DEC_HEADS = 16
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

Create the encoder and decoder

In [ ]:
enc = Encoder(INPUT_DIM, HID_DIM, ENC_LAYERS, ENC_HEADS,
              ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HID_DIM, DEC_LAYERS,
              DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

Set the padding index for the source and target sequences

In [ ]:
SRC_PAD_IDX = PAD_token
TRG_PAD_IDX = PAD_token

Create the Seq2Seq model

In [ ]:
model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

Function to count trainable parameters in the model

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

Print the number of trainable parameters in the model

In [ ]:
print(f'The model has {count_parameters(model):,} trainable parameters')

Function to initialize weights in the model

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

Apply weight initialization to the model

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
model.apply(initialize_weights)

Define the learning rate

In [ ]:
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Define the optimizer

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Define the loss function (cross-entropy loss ignoring padding index)

In [ ]:
class CrossEntropyLoss(nn.CrossEntropyLoss):
    """CrossEntropyLoss - with ability to receive distribution as targets, and optional label smoothing"""
    def __init__(self, weight=None, ignore_index=-100, reduction='mean', smooth_eps=None, smooth_dist=None, from_logits=True):
        super(CrossEntropyLoss, self).__init__(weight=weight,
                                               ignore_index=ignore_index, reduction=reduction)
        self.smooth_eps = smooth_eps
        self.smooth_dist = smooth_dist
        self.from_logits = from_logits
    def forward(self, input, target, smooth_dist=None):
        if smooth_dist is None:
            smooth_dist = self.smooth_dist
        return cross_entropy(input, target, weight=self.weight, ignore_index=self.ignore_index,
                             reduction=self.reduction, smooth_eps=self.smooth_eps,
                             smooth_dist=smooth_dist, from_logits=self.from_logits)

In [ ]:
def cross_entropy(inputs, target, weight=None, ignore_index=-100, reduction='mean',
                  smooth_eps=None, smooth_dist=None, from_logits=True):
 # """Cross entropy loss, with support for target distributions and label smoothing https://arxiv.org/abs/1512.00567"""
    smooth_eps = smooth_eps or 0

    # Ordinary log-likelihood - use cross_entropy from nn
    if _is_long(target) and smooth_eps == 0:
        if from_logits:
            return F.cross_entropy(inputs, target, weight, ignore_index=ignore_index, reduction=reduction)
        else:
            return F.nll_loss(inputs, target, weight, ignore_index=ignore_index, reduction=reduction)
    if from_logits:
        # Log-softmax of inputs
        lsm = F.log_softmax(inputs, dim=-1)
    else:
        lsm = inputs
    masked_indices = None
    num_classes = inputs.size(-1)
    if _is_long(target) and ignore_index >= 0:
        masked_indices = target.eq(ignore_index)
    if smooth_eps > 0 and smooth_dist is not None:
        if _is_long(target):
            target = onehot(target, num_classes).type_as(inputs)
        if smooth_dist.dim() < target.dim():
            smooth_dist = smooth_dist.unsqueeze(0)
        target.lerp_(smooth_dist, smooth_eps)
    if weight is not None:
        lsm = lsm * weight.unsqueeze(0)
    if _is_long(target):
        eps_sum = smooth_eps / num_classes
        eps_nll = 1. - eps_sum - smooth_eps
        likelihood = lsm.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1)
        loss = -(eps_nll * likelihood + eps_sum * lsm.sum(-1))
    else:
        loss = -(target * lsm).sum(-1)
    if masked_indices is not None:
        loss.masked_fill_(masked_indices, 0)
    if reduction == 'sum':
        loss = loss.sum()
    elif reduction == 'mean':
        if masked_indices is None:
            loss = loss.mean()
        else:
            loss = loss.sum() / float(loss.size(0) - masked_indices.sum())
    return loss

In [ ]:
def onehot(indexes, N=None, ignore_index=None):
    """
    Creates a one-representation of indexes with N possible entries
    if N is not specified, it will suit the maximum index appearing.
    indexes is a long-tensor of indexes
    ignore_index will be zero in onehot representation
    """
    if N is None:
        N = indexes.max() + 1
    sz = list(indexes.size())
    output = indexes.new().byte().resize_(*sz, N).zero_()
    output.scatter_(-1, indexes.unsqueeze(-1), 1)
    if ignore_index is not None and ignore_index >= 0:
        output.masked_fill_(indexes.eq(ignore_index).unsqueeze(-1), 0)
    return output

In [ ]:
def _is_long(x):
    if hasattr(x, 'data'):
        x = x.data
    return isinstance(x, torch.LongTensor) or isinstance(x, torch.cuda.LongTensor)

In [ ]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = CrossEntropyLoss(ignore_index=TRG_PAD_IDX, smooth_eps=0.20)
    loss = crossEntropy(inp, target)
    loss = loss.to(device)
    return loss, nTotal.item()

Replace TRG_PAD_IDX with the actual index of the padding token in your target vocabulary

In [ ]:
TRG_PAD_IDX = vocab['<PAD>']  # The padding index for output (CPP code)
criterion = maskNLLLoss

Function to create the target mask

In [ ]:
def make_trg_mask(trg):
    trg_pad_mask = (trg != TRG_PAD_IDX).unsqueeze(1).unsqueeze(2)
    trg_len = trg.shape[1]
    trg_sub_mask = torch.tril(torch.ones(
        (trg_len, trg_len), device=device)).bool()
    trg_mask = trg_pad_mask & trg_sub_mask
    return trg_mask

Your custom maskNLLLoss function<br>
...

Training function

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    n_totals = 0
    print_losses = []
    for batch in tqdm(iterator, total=len(iterator)):
        src = batch[0].permute(1, 0)  # Access the input sequence
        trg = batch[1].permute(1, 0)  # Access the output sequence
        trg_mask = make_trg_mask(trg)
        optimizer.zero_grad()
        output, _ = model(src, trg[:, :-1])
        output_dim = output.shape[-1]
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:, 1:].contiguous().view(-1)
        mask_loss, nTotal = criterion(output, trg, trg_mask)
        mask_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        print_losses.append(mask_loss.item() * nTotal)
        n_totals += nTotal
    return sum(print_losses) / n_totals

Evaluate Function

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    n_totals = 0
    print_losses = []
    with torch.no_grad():
        for batch in tqdm(iterator, total=len(iterator)):
            src = batch[0].permute(1, 0)  # Access the input sequence
            trg = batch[1].permute(1, 0)  # Access the output sequence
            trg_mask = make_trg_mask(trg)
            output, _ = model(src, trg[:, :-1])
            output_dim = output.shape[-1]
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:, 1:].contiguous().view(-1)
            mask_loss, nTotal = criterion(output, trg, trg_mask)
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    return sum(print_losses) / n_totals

In [ ]:
model_path = os.path.join(os.path.dirname(
    os.path.abspath(__file__)), 'model.pt')

Training Loop

In [ ]:
def load_checkpoint(model, optimizer, checkpoint_file):
    checkpoint = torch.load(checkpoint_file)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    best_valid_loss = checkpoint['loss']
    return epoch, best_valid_loss

In [ ]:
def save_checkpoint(model, optimizer, epoch, loss, filename):
    checkpoint = {
        'epoch': epoch+1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }

    # After each epoch or a specific number of iterations
    torch.save(checkpoint, 'checkpoint.pt')

In [ ]:
checkpoint_file = 'checkpoint.pt'
if os.path.exists(checkpoint_file):
    epoch, best_valid_loss = load_checkpoint(model, optimizer, checkpoint_file)
    print(
        f"Resuming training from epoch {epoch + 1} with best validation loss: {best_valid_loss:.3f}")
else:
    epoch = 0
    best_valid_loss = float('inf')

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 3
CLIP = 1
best_valid_loss = float('inf')

In [ ]:
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_loader, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(
            'https://drive.google.com/drive/u/0/folders/1ERMKFGQffkopZMlLtIUKDbHG6XhOfmdW', 'ourmodel.pt')
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(
        f'\tVal. Loss: {valid_loss:.3f} | Val. PPL: {math.exp(valid_loss):7.3f}')